In [493]:
%matplotlib inline

In [494]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [495]:
import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



## 1. Read Compustat Data

In [496]:
compustat_data = pd.read_csv("compustat_data_new.csv")[['LPERMNO','LPERMCO','fyear','seq']]

In [497]:
compustat_data.head()

,LPERMNO,LPERMCO,fyear,seq
0,25881,23369,1970,10.544
1,25881,23369,1971,8.382
2,25881,23369,1972,7.021
3,25881,23369,1973,8.567
4,25881,23369,1974,10.257


In [498]:
compustat_data.isnull().sum()

LPERMNO        0
LPERMCO        0
fyear          1
seq        28188
dtype: int64

In [499]:
compustat_data.dropna(subset=['fyear', 'seq'], how="any",inplace=True)

In [500]:
compustat_data.sort_values(by='fyear', inplace=True)

If we see the last row of the below table, we can see that the equity changes for different LPERMNO for the 
same company. So consider that LPERMNO, LPERMCO as the combination that captures the Book/Market Ratio

In [501]:
compustat_data[compustat_data.LPERMCO == 45301].tail(15)

,LPERMNO,LPERMCO,fyear,seq
271644,90228,45301,2011,2805.4
271648,90866,45301,2012,2210.0
271649,90227,45301,2012,2210.0
271647,90228,45301,2012,2210.0
271652,90227,45301,2013,12025.8
271650,90228,45301,2013,12025.8
271651,90866,45301,2013,12025.8
271655,90227,45301,2014,14714.5
271653,90228,45301,2014,14714.5
271654,90866,45301,2014,14714.5


In [502]:
def get_prev_book_value(series):
    prev_year = series['fyear']
    shifted_series = series.shift(1)
    correct_rows = (prev_year == (shifted_series['fyear']+1))
    return shifted_series.ix[correct_rows,'seq']

In [503]:
compustat_data['prev_seq'] = compustat_data.groupby(['LPERMCO','LPERMNO'],group_keys = False).apply(get_prev_book_value)

In [504]:
compustat_data[compustat_data.LPERMCO == 22287]

,LPERMNO,LPERMCO,fyear,seq,prev_seq
35863,11543,22287,1960,39.110,NaN
35864,11543,22287,1961,36.379,39.110
35865,11543,22287,1962,35.485,36.379
35866,11543,22287,1963,29.701,35.485
35867,11543,22287,1964,31.454,29.701
35868,11543,22287,1965,12.142,31.454
35869,11543,22287,1966,14.178,12.142
35870,11543,22287,1967,20.958,14.178
35871,11543,22287,1968,38.601,20.958


### Delete those rows with no prev year seq data

In [505]:
compustat_data.dropna(subset=['prev_seq'],inplace=True)

In [506]:
compustat_data.drop('prev_seq', axis = 1, inplace=True)

In [507]:
compustat_data.rename(columns = {"LPERMCO":"PERMCO","LPERMNO":"PERMNO"}, inplace=True)

In [508]:
compustat_data.head()

,PERMNO,PERMCO,fyear,seq
82374,19043,22890,1960,199.898
58661,25160,20847,1961,65.602
82375,19043,22890,1961,211.436
35864,11543,22287,1961,36.379
140592,19940,22960,1961,249.852


# 2. CRSP Data

In [ ]:
crsp_data = pd.read_csv("crsp_data_new.csv",parse_dates=['date'])

In [510]:
crsp_data.head(2)

,PERMNO,date,SHRCD,SICCD,TICKER,COMNAM,PERMCO,CUSIP,DLRETX,DLRET,PRC,RET,SHROUT,RETX
0,10000,1986-01-31,10,3990,OMFGA,OPTIMUM MANUFACTURING INC,7952,68391610,NaN,NaN,-4.375,C,3680,C
1,10000,1986-02-28,10,3990,OMFGA,OPTIMUM MANUFACTURING INC,7952,68391610,NaN,NaN,-3.250,-0.257143,3680,-0.257143


In [511]:
crsp_data.dtypes

PERMNO             int64
date      datetime64[ns]
SHRCD              int64
SICCD             object
TICKER            object
COMNAM            object
PERMCO             int64
CUSIP             object
DLRETX            object
DLRET             object
PRC              float64
RET               object
SHROUT           float64
RETX              object
dtype: object

In [512]:
crsp_data.isnull().sum()

PERMNO          0
date            0
SHRCD           0
SICCD           0
TICKER     220551
COMNAM          0
PERMCO          0
CUSIP           0
DLRETX    3225637
DLRET     3225637
PRC         90482
RET         39723
SHROUT       4443
RETX        39723
dtype: int64

### Calculating Year and month from the data

In [513]:
crsp_data['year'] = crsp_data.date.dt.year
crsp_data['month'] = crsp_data.date.dt.month

## CRSP Data Cleaning

### a. Convert the Price to its absolute value

In [514]:
crsp_data['PRC'] = crsp_data.PRC.abs()

### b. Convert the Ret , Retx , DLRet , DLRetx which have codes to NaN

In [515]:
crsp_data['DLRET'] = pd.to_numeric(crsp_data.DLRET,errors='coerce')
crsp_data['RET'] = pd.to_numeric(crsp_data.RET, errors = 'coerce')
crsp_data['DLRETX'] = pd.to_numeric(crsp_data.DLRETX,errors='coerce')
crsp_data['RETX'] = pd.to_numeric(crsp_data.RETX, errors = 'coerce')

In [516]:
crsp_data.isnull().sum()

PERMNO          0
date            0
SHRCD           0
SICCD           0
TICKER     220551
COMNAM          0
PERMCO          0
CUSIP           0
DLRETX    3229538
DLRET     3229538
PRC         90482
RET        114897
SHROUT       4443
RETX       114897
year            0
month           0
dtype: int64

Number of NaN s in Ret and RetX increases because it used to have alphabet codes when the company gets delisted. 

### Where Ret is not available, copy DLRet. Same is the case for RetX, DLRetX pair

In [517]:
crsp_data.ix[crsp_data.DLRET.notnull(),'RET'] = crsp_data.ix[crsp_data.DLRET.notnull(),'DLRET']
crsp_data.ix[crsp_data.DLRETX.notnull(),'RETX'] = crsp_data.ix[crsp_data.DLRETX.notnull(),'DLRETX']

In [518]:
crsp_data.isnull().sum()

PERMNO          0
date            0
SHRCD           0
SICCD           0
TICKER     220551
COMNAM          0
PERMCO          0
CUSIP           0
DLRETX    3229538
DLRET     3229538
PRC         90482
RET         97272
SHROUT       4443
RETX        97272
year            0
month           0
dtype: int64

### Calculate the Market Cap for the rows of data


In [519]:
crsp_data['market_cap'] = (crsp_data['SHROUT'] * crsp_data['PRC'])/1000 

### For each row the recent fiscal year is current_year minus one. 

In [520]:
crsp_data['fyear'] = crsp_data['year']- 1

#### Check for duplicate rows

In [521]:
crsp_data.duplicated(subset=['PERMCO','PERMNO','date']).sum()

0

### Getting Market Cap for each row in Compustat by merging with CRSP. 
### For each fyear data, get the market cap in december same year
#### Merging on PERMCO,PERMNO, year basis

In [522]:
compustat_data = pd.merge(compustat_data, crsp_data.ix[crsp_data.month == 12, ['PERMCO','PERMNO','year','market_cap']],
                         left_on = ['PERMCO','PERMNO','fyear'], right_on = ['PERMCO','PERMNO','year'], suffixes = ['_comp','_crsp'],
                         how = 'left')

In [523]:
compustat_data.isnull().sum()

PERMNO            0
PERMCO            0
fyear             0
seq               0
year          29466
market_cap    32212
dtype: int64

### some rows don't have year data this means for those rows, market cap data is not available. So delete that data. 

In [524]:
compustat_data.dropna(subset=['year'], inplace=True)

In [525]:
compustat_data.isnull().sum()

PERMNO           0
PERMCO           0
fyear            0
seq              0
year             0
market_cap    2746
dtype: int64

Most of the above market_cap NaN rows are from December 2015. Because the downloaded dataset is upto dec 2015. Many 
companies does not have Price data, so market cap is not available. Drop these rows as well.

In [526]:
compustat_data.dropna(subset=['market_cap'], inplace=True)

In [527]:
compustat_data['BMR'] = compustat_data.seq/compustat_data.market_cap

In [528]:
compustat_data.head()

,PERMNO,PERMCO,fyear,seq,year,market_cap,BMR
0,19043,22890,1960,199.898,1960,260.77500,0.766554
1,25160,20847,1961,65.602,1961,57.72000,1.136556
2,19043,22890,1961,211.436,1961,362.01850,0.584048
3,11543,22287,1961,36.379,1961,19.36675,1.878426
4,19940,22960,1961,249.852,1961,491.73900,0.508099


### Merge CRSP Data with the Compustat Data to get the corresponding fyear BMR 
#### Merge Using Company Number , Financial year ,PERMNO

In [529]:
crsp_data = pd.merge(crsp_data, compustat_data[['PERMCO','PERMNO','fyear','BMR']],
         left_on = ['PERMCO','PERMNO', 'fyear'], right_on =['PERMCO','PERMNO', 'fyear'], how = 'left' )

# 3. Creating Portfolio Weights at the end of June

### Get the BMR and Marketcap for all the stocks at the end of june

In [530]:
june_data = crsp_data.ix[crsp_data.month == 6, ['PERMNO','PERMCO','year','market_cap','BMR']]

### Delete those rows which does not have Market cap and BMR

In [531]:
june_data.dropna(subset=['market_cap','BMR'], how = 'any', inplace=True)

In [532]:
june_data.duplicated(subset= ['PERMCO','PERMNO','year']).sum()

0

#### No duplicate rows in the above dataset

In [533]:
june_data.isnull().sum()

PERMNO        0
PERMCO        0
year          0
market_cap    0
BMR           0
dtype: int64

## Calculating Size here
#### ranks are in ascending order here

In [534]:
june_data['market_cap_percentile'] = june_data[['year',
                                                'market_cap']].groupby('year').rank(pct=True)['market_cap']

In [535]:
june_data['Size'] = np.where(june_data['market_cap_percentile']>=0.8, "Big",'Small')

In [536]:
june_data.head()

,PERMNO,PERMCO,year,market_cap,BMR,market_cap_percentile,Size
47,10001,7953,1988,6.20000,0.921242,0.157191,Small
59,10001,7953,1989,7.00700,0.866360,0.175465,Small
71,10001,7953,1990,10.05225,0.610857,0.237179,Small
83,10001,7953,1991,11.26650,0.716968,0.267385,Small
95,10001,7953,1992,12.63125,0.512077,0.233716,Small


## Calculating Value here 

In [537]:
june_data['Value_percentile']=june_data[['year','Size','BMR']].groupby(['year','Size']).rank(pct = True)['BMR']

In [538]:
june_data['Value']  = np.nan

In [539]:
june_data.ix[june_data.Value_percentile <= 0.30,'Value'] = "Growth"
june_data.ix[june_data.Value_percentile >= 0.70,'Value'] = "Value"
june_data.ix[np.logical_and(june_data.Value_percentile < 0.70,
                            june_data.Value_percentile > 0.30) ,'Value'] = 'Neutral'

### Verifying the percentiles for 2015 year

In [540]:
pd.pivot_table(june_data[june_data.year == 2015], index=['Size','Value'],aggfunc=len,margins=True)

BMR  PERMCO  PERMNO  Value_percentile  market_cap  market_cap_percentile  year
Size  Value                                                                                   
Big   Growth    201     201     201               201         201                    201   201
      Neutral   267     267     267               267         267                    267   267
      Value     202     202     202               202         202                    202   202
Small Growth    802     802     802               802         802                    802   802
      Neutral  1070    1070    1070              1070        1070                   1070  1070
      Value     803     803     803               803         803                    803   803
All            3345    3345    3345              3345        3345                   3345  3345

In [541]:
june_data = june_data.ix[:,['PERMNO','PERMCO','year','Size','Value','market_cap']]

## Calculating the value weighted portfolio here

In [542]:
june_data['port_weights'] =  june_data[['year','Size','Value',
                  'market_cap']].groupby(['year',
                'Size','Value'],group_keys = False).apply(lambda x: x['market_cap']/(x['market_cap'].sum()))

### Verifying the portfolio weights here for all the portfolio for year 2015

In [543]:
pd.pivot_table(june_data[(june_data.year == 2015 ) ],index=['Size','Value'],aggfunc=np.sum,margins=True)['port_weights']

Size   Value  
Big    Growth     1
       Neutral    1
       Value      1
Small  Growth     1
       Neutral    1
       Value      1
All               6
Name: port_weights, dtype: float64

## Merge with CRSP

#### Creating Portfolio year and Portfolio month here

portfolio year is the year when the portfolio weights were decided. 
portfolio month is the month from the constution of the portfolio. 

In [544]:
crsp_data['portfolio_year'] = np.nan
crsp_data.ix[crsp_data.month > 6,'portfolio_year'] = crsp_data.ix[crsp_data.month > 6,'year']
crsp_data.ix[crsp_data.month <= 6,'portfolio_year'] = crsp_data.ix[crsp_data.month <= 6,'year'] - 1

In [545]:
crsp_data['portfolio_month'] = np.nan
crsp_data.ix[crsp_data.month > 6,'portfolio_month'] = crsp_data.ix[crsp_data.month > 6,'month'] -6 
crsp_data.ix[crsp_data.month <= 6,'portfolio_month'] = crsp_data.ix[crsp_data.month <= 6,'month'] +6

In [546]:
crsp_data = pd.merge(crsp_data, june_data[['PERMCO','PERMNO','year','Size','Value','port_weights']],
        left_on = ['PERMCO','PERMNO','portfolio_year'],
        right_on = ['PERMCO','PERMNO','year'],how = "left")

#### Delete those rows for which port_weights are not available

In [547]:
crsp_data.dropna(subset=['port_weights'],inplace=True)

In [548]:
crsp_data.isnull().sum()

PERMNO                   0
date                     0
SHRCD                    0
SICCD                    0
TICKER                6966
COMNAM                   0
PERMCO                   0
CUSIP                    0
DLRETX             2203658
DLRET              2203658
PRC                  20313
RET                   8018
SHROUT                1821
RETX                  8018
year_x                   0
month                    0
market_cap           20313
fyear                    0
BMR                  11266
portfolio_year           0
portfolio_month          0
year_y                   0
Size                     0
Value                    0
port_weights             0
dtype: int64

In [549]:
crsp_data[crsp_data.BMR.isnull()]['portfolio_month'].unique()

array([  7.,   8.,   9.,  10.,  11.,  12.])

The NaN in market_cap and BMR is due to fact that they are not availabe for last fiscal year. Whereas the stocks are included last year, which used the previous year stats to include in the portfolio. 

Delete those rows which do not have Returns

In [550]:
crsp_data.dropna(subset = ['RET'], inplace=True)

In [551]:
crsp_data.isnull().sum()

PERMNO                   0
date                     0
SHRCD                    0
SICCD                    0
TICKER                3034
COMNAM                   0
PERMCO                   0
CUSIP                    0
DLRETX             2195640
DLRET              2195640
PRC                  12451
RET                      0
SHROUT                   0
RETX                     0
year_x                   0
month                    0
market_cap           12451
fyear                    0
BMR                   8723
portfolio_year           0
portfolio_month          0
year_y                   0
Size                     0
Value                    0
port_weights             0
dtype: int64

## Your portfolio weights change every month because of the last month returns

In [552]:
port_data = crsp_data.ix[:,['PERMNO','PERMCO','RET','port_weights','portfolio_year','portfolio_month']]

Sort by Portfolio year and month before using groupby and shift

In [553]:
port_data.sort_values(by = ['PERMCO', 'PERMNO','portfolio_year', 'portfolio_month'], inplace=True)

In [554]:
port_data['del_ret'] = port_data.groupby(['PERMCO','PERMNO','portfolio_year',])['RET'].shift(1)+1

In [555]:
port_data.ix[port_data.portfolio_month == 1,'del_ret'] = 1 

In [556]:
port_data['cum_ret'] = port_data.groupby(['PERMCO','PERMNO','portfolio_year'])['del_ret'].cumprod()

In [557]:
port_data.dropna(subset=['cum_ret'], inplace=True)

## Merge cum_ret of port_data with crsp_data

In [558]:
crsp_data = pd.merge(crsp_data, port_data[['PERMNO','PERMCO','portfolio_year','portfolio_month','cum_ret']])

In [559]:
crsp_data.dropna(subset =['port_weights','cum_ret','market_cap'], inplace=True)

In [560]:
crsp_data['port_weights'] = crsp_data.port_weights *crsp_data.cum_ret

In [619]:
penul_data = crsp_data.groupby(['year_x','month','Size',
                          'Value'])[['RET','port_weights']].apply(lambda g:np.average(g['RET'], weights=g['port_weights']))

In [620]:
ultimate = penul_data.unstack('Size').unstack('Value').reset_index().sort_values(by=['year_x','month'])

In [621]:
ultimate.columns = [' '.join(col).strip() for col in ultimate.columns.values]

In [622]:
ultimate['SMB'] =1/3*(ultimate['Small Growth']+ultimate['Small Value']+ultimate['Small Neutral']) -\
                1/3*(ultimate['Big Growth']+ultimate['Big Value']+ultimate['Big Neutral'])

In [623]:
ultimate['SMB_Perc'] = ultimate['SMB']*100

In [625]:
ultimate['HML'] = (0.5*(ultimate['Big Value']+ultimate['Small Value']) - \
                   0.5*(ultimate['Big Growth']+ultimate['Small Growth'])  )*100

In [626]:
crsp_data.cum_ret.isnull().sum()

0

In [627]:
ultimate[ultimate.year_x>=1970].to_csv("factor.csv")

In [632]:
ultimate.ix[ultimate.year_x == 1970]

,year_x,month,Big Growth,Big Neutral,Big Value,Small Growth,Small Neutral,Small Value,SMB,SMB_Perc,HML
102,1970,1,-0.079762,-0.090284,-0.064485,-0.073181,-0.070267,-0.037066,0.018006,1.800561,2.569600
103,1970,2,0.038192,0.077874,0.079642,0.035726,0.044845,0.064075,-0.017021,-1.702116,3.489987
104,1970,3,-0.018210,0.014748,0.013263,-0.036278,0.001339,0.015010,-0.009910,-0.991000,4.138075
105,1970,4,-0.094792,-0.091676,-0.075648,-0.194109,-0.130785,-0.102539,-0.055106,-5.510593,5.535689
106,1970,5,-0.073884,-0.038998,-0.038491,-0.102615,-0.098851,-0.071959,-0.040684,-4.068418,3.302483
107,1970,6,-0.058805,-0.028558,-0.061782,-0.058147,-0.056802,-0.064418,-0.010074,-1.007436,-0.462415
108,1970,7,0.042988,0.097625,0.091686,0.082947,0.074833,0.082921,0.002800,0.280031,2.433623
109,1970,8,0.039644,0.055627,0.046372,0.071198,0.069235,0.059200,0.019330,1.933010,-0.263560
110,1970,9,0.068748,0.006237,0.012499,0.128817,0.081692,0.078055,0.067027,6.702704,-5.350519
111,1970,10,0.000171,-0.007188,-0.021429,-0.043978,-0.054344,-0.053686,-0.041187,-4.118715,-1.565358


In [631]:
1/3*(-0.082037-0.052705-0.081341) - 1/3*(-0.063200-0.020646-0.027509)

-0.03490933333333332

In [637]:
compustat_data[compustat_data.PERMNO == 16598]

,PERMNO,PERMCO,fyear,seq,year,market_cap,BMR
2973,16598,22702,1967,125.778,1967,531.247500,0.236760
4457,16598,22702,1968,122.293,1968,474.609000,0.257671
6002,16598,22702,1969,123.956,1969,258.589125,0.479355
6823,16598,22702,1970,129.407,1970,280.560750,0.461244
9161,16598,22702,1971,156.314,1971,263.879000,0.592370
10812,16598,22702,1972,160.126,1972,203.333250,0.787505
12619,16598,22702,1973,164.346,1973,105.233875,1.561721
17387,16598,22702,1974,183.280,1974,103.450250,1.771673
21900,16598,22702,1975,194.685,1975,140.906375,1.381662
25765,16598,22702,1976,201.235,1976,165.877125,1.213157


In [638]:
crsp_data[crsp_data.PERMNO == 16598]

,PERMNO,date,SHRCD,SICCD,TICKER,COMNAM,PERMCO,CUSIP,DLRETX,DLRET,PRC,RET,SHROUT,RETX,year_x,month,market_cap,fyear,BMR,portfolio_year,portfolio_month,year_y,Size,Value,port_weights,cum_ret
253890,16598,1968-07-30,10,3357,GK,GENERAL CABLE CORP,22702,36176510,NaN,NaN,33.000,-0.098976,13492,-0.098976,1968,7,445.236000,1967,0.236760,1968,1,1968,Big,Growth,0.002902,1.000000
253891,16598,1968-08-30,10,3357,GK,GENERAL CABLE CORP,22702,36176510,NaN,NaN,35.875,0.087121,13492,0.087121,1968,8,484.025500,1967,0.236760,1968,2,1968,Big,Growth,0.002615,0.901024
253892,16598,1968-09-30,10,3357,GK,GENERAL CABLE CORP,22702,36176510,NaN,NaN,36.875,0.036237,13492,0.027875,1968,9,497.517500,1967,0.236760,1968,3,1968,Big,Growth,0.002842,0.979522
253893,16598,1968-10-31,10,3357,GK,GENERAL CABLE CORP,22702,36176510,NaN,NaN,38.000,0.030508,13492,0.030508,1968,10,512.696000,1967,0.236760,1968,4,1968,Big,Growth,0.002945,1.015017
253894,16598,1968-11-29,10,3357,GK,GENERAL CABLE CORP,22702,36176510,NaN,NaN,39.500,0.039474,13492,0.039474,1968,11,532.934000,1967,0.236760,1968,5,1968,Big,Growth,0.003035,1.045983
253895,16598,1968-12-31,10,3357,GK,GENERAL CABLE CORP,22702,36176510,NaN,NaN,35.125,-0.103165,13512,-0.110759,1968,12,474.609000,1967,0.236760,1968,6,1968,Big,Growth,0.003155,1.087272
253896,16598,1969-01-31,10,3357,GK,GENERAL CABLE CORP,22702,36176510,NaN,NaN,29.500,-0.160142,13512,-0.160142,1969,1,398.604000,1968,0.257671,1968,7,1968,Big,Growth,0.002830,0.975104
253897,16598,1969-02-28,10,3357,GK,GENERAL CABLE CORP,22702,36176510,NaN,NaN,29.375,-0.004237,13512,-0.004237,1969,2,396.915000,1968,0.257671,1968,8,1968,Big,Growth,0.002376,0.818949
253898,16598,1969-03-28,10,3357,GK,GENERAL CABLE CORP,22702,36176510,NaN,NaN,26.500,-0.087660,13512,-0.097872,1969,3,358.068000,1968,0.257671,1968,9,1968,Big,Growth,0.002366,0.815479
253899,16598,1969-04-30,10,3357,GK,GENERAL CABLE CORP,22702,36176510,NaN,NaN,27.000,0.018868,13512,0.018868,1969,4,364.824000,1968,0.257671,1968,10,1968,Big,Growth,0.002159,0.743994
